There are many different kinds of data models, and every data model embodies assumptions about how it is going to be used. Some kinds of usage are easy and some are not supported; some operations are fast and some perform badly; some data transformations feel natural and some are awkard.

It can take a lot of effort to master just one data model (think how many books there are on relational data modeling). Building software is hard enough, even when working with just one data model and without worrying about its inner working. But since the data model has such a profound effect on what the software above it can and can't to, it's important to choose one that is appropriate to the application.

In this chapter we will look at a range of general-purpose data models for data storage and querying. In particular, we will compare the relational model, the cocument model, and a few graph-based data models. We will also look at various query languages and compare their use cases. In Chapter 3 we will discuss how storage engines work; that is, how these data models are actually implemented.

## Relational Model Versus Document Model

The best-known data model today is probably that of SQL, based on the relational model proposed by Edgar Codd in 1970 [1]: data is organized into relations (called tables in SQL), where each relation is an unordered collection of tuples. 

The relational model was a theoretical proposal, and many people at the time doubted whether it could be implemented efficiently. However, by the mid-1980s, relational database management systems and SQL had some kind of regular structure. The dominance of relational databases has lasted around 25 - 30 years an eternity in computing history.

The roots of relational databases lie in business data processing, which was performed on mainframe computers in the 1960s and '70s. The use cases appear mundane from today's perspective: typically transaction processing (entering sales or banking transactions, airline reservation

Other databases at that time forced application developers to think a lot about the internal representation of the data in the databse. The goal of the relational model was to hide that implementation detail behind a cleaner interface.

Over the years, there have been many competing approaches to data storage and querying. IN the 1970s and early 1980s, the network model and the hierarchical model were the main alternatives, but the relational model came to dominate them. Object databses came and went again in the late 1980s and early 1990s. XML databses appeared in the early 2000s, but have only seen niche adoption. Each competitor to the relational model generated a lot of hype in its time, but it never lasted.

As computers became vasyly more powerful and networked, they started being used for increasingly diverse purposes. and remarkably, relational databses turned out to generalize very well, beyond their original socpe of bsiness data processing, to a broad variety of use cases. Much of what you see on the web today is still powered by relational databses, be it online publishing, discussion, social networking, ecommerce, games, software-as-a-service productibity applications, or much more.

The Birth of NoSQL

Now, in the 2010s, NoSQL is the latest attempt to overthrow the relational model's domiance. The name "NoSQL" is unfortunate, since it doesn't actually refer to any particular technology- it was originally intended simply as a catchy Twitter hashtag for a meetup on open source, distributed, nonrelational databases in 2009.

## The Object-Relational Mismatch

Most application development today is done in object-orieented programming languages, which leads to a common criticism of the SQLdata model; if data is stored in relational tables, an awkward translation layer is required between the objects in the application code and the databse model of tables, rows, and columns. The disconnect between the models is sometimes called an impedance mismatch.

Object-relational mapping frameworks like ActiveRecord and Hibernate reduce the amount of boilerplate code required for this translation layer, but they can't completely hide the differences between the two models.

For example, Figure 2-1 illustrates how a resume could be expressed in a relational schema. The profile as a whole can be identified by a unique identifier, user_id. Fields like first_name and last_name appear exactly once per user, so they can be modeled as columns on the users table. However, most people have had more than one job in their career, and people may have varying numbers of perios of education and any number of pieces of contact information. There is a one-to-many relationship from the user to these items, which can be represented in various ways:

- In the traditional SQL model, the most common normalized representation is to put positions, education, and contac information in separate tables, with a foreign key reference to the users table, as in Figure 2-1
- Later versions of the SQL standard added support for structued datatypes and XML data; this allowed multi-valued data to be stored within a single row, with support for querying and indexing inside those documents. These features are supported to vargin degrees by Oracle, IBM DB2, MS SQL Server, adn PostgreSQL. A JSON datatype is also supported by several databases, including IBM DB2, MySQL, and PostgreSQL.

- A third option is to encode jobs, education, and contact info as a JSON or XML document, store it on a text column in the databses, and let the application interpret its structure and content. IN this setup, you typically cannot use the databse to query for values inside that encoded column.

Some developers feel that JSON model reduces the impedance mismatch between the application code and the storage layer. However, as we shall see in Chapter 4, there are also problems with JSON as a data encoding format. The lack of a schema is often cited as an advantage; we will discuss this in Schema flexibility in the document model on page 39

The JSON representation has better locality than the multi-table schema, in Figure 2-1. If you want to fetch a profile in the relational example, you need to either perform multiple queries (query each table by user_id) or perform a messy multi-way join between the users table and its subordinate tables. In the JSON representation, all the relevant information is on one place, and one query is sufficient.

The one-to-many relationships from the user profile to the user's positions, educational history, and contact information imply a tree structure in the data, and the JSON representation makes this tree strcuture explicit

Unfortunately, normalizing this data requires many-to-one relationships, which don't fit nicely into the document model. In relational databses, it's normal to refer to rows in other tables by ID, because joins are easy. In document databses, joins are not needed for one-to-many tree structures, and support for joins is often weak.

If the databse itself does not support joins, you have to emulate a join in application code by making multiple queries to the databse. 

Moreover, even if the initial version of an application ifts well in a join-free document model, data has a tendency of becoming more interconnected as features are added to applications. For example, consider some changes we could make to the resume example.

- Organizations and schools as entities 
In the previous description, organization are just strings. Perhaps they should be references to entities instead? Then each organization, school, or university could have its own web page; each resume could link to the organizations and schools that it mentions, and include their logos and other information

- Recommendations
Say you want tot add a new feature: one user can write a recommendation for another user. The recommendation is shown on the resume of the user who was recommended, together with the name and photo of the user making the recommendation. If the recommender updates their photo, any recommendations they have written need to reflect the new photo. Therefore, the recommendation should have a reference to the author's profile.

## Are Document Databases Repeating History?

While many-to-many relationships and joins are routinely used in relational databses, document databses and NoSQL reopened the debate on how best to represent such relationships in a database. This debate is much older than NoSQL - infact, it goes back to the very earliest computerized database systems.

The most popular databses for business data processing in the 1970s was IBM's INformation Management System, originally developed for stock-keeping in the Apollo space program and first commercially released in 1968. It is stiil in use and maintained today, running on OS/390 on IBM mainframes.

The design of IMS used a fairly simple data model called the hierarchical model, which has some remarkable similarities to the JSON model used by cosument databses . It represented all data as a tree of records nested within records, much like the JSON structure.

If the same query can be written 4 lines in one query language but requires 29 lines in another, that just shows that different data models are designed to satisfy different use cases. It's important to pick a data model that is suitable for your application.

Triple-Stores and SPARQL

#### The semantic web

If you read more about triple-stores, you many get sucked into a maelstrom of articles written about the semantic web. The trile-store data model is completely independent of the semantic web-for example, Datomic is a triple-store that does not claim to have anything to do with it. But since the two wre so closely linked in many people's minds, we should discuss them briefly.

The semantic web is fundamentally a simple and reasonable idea: websites already publish information as text and pictures for humans to read, so why don't they also publish information as machine-readable data for computers to read? The Resource Description Framework was intended as a mechanism for different websites to publish data in a consistent format, allowing data from different websites to be automatically combined into a web of data - a kindd of internet -wide "databse of everything"

# Storage and Retrieval

On the most fundamental level, a database needs to do two things: when you igve it some data, it should store the data, and then you ask it again later, it should give the data back to you.

In Chapter 2 we discussed data models and query languages-i.e., the format in which you give the database your data, and the mechanism by which you can ask for it again later. In this chapter we discuss the same from the database's point of view: how we can store the data that we're given , and how we can find it again when we're asked for it.

Why should you, as an application developer care how the databse handles storage and retrieval internally?

An index is an additional structure that is derived from the primary data. Many databases allow you to add and remove indexes, and this doesn't affect the contents of the databse; it only affects the performance of queries. Maintaining additional structures incurs overhead, especailly on writes. FOor writes, it's hard to beat the performance of simply appending to a file, because that's the simplest possible write operation. Any kind of index usually slows down writes, because the index also needs to be updated every time data is written.

This is an important trade-off in storage systems: well-chosen indexes speed up read queries, but every index slows down writes. For this reason, databses don't usually index everything by default, but require you - the application developer or databse administrator - to choose indexes manually, using your knowledge of the application's typical query patterns. You can then choose the indexes that give your application the greatest benefit, without introducing more overhead than necessary.

## Hash indexes

Let's start with indexes for key-value data. This is not the only kind of data you can index, but it's very common, and it's a useful building block for more complex indexes.

Key-value stores are quite similar to the dictionary type that you can find in most programming languages, and which is usually implemented as a hash map.

We can then perform compaction on these segments, as illustrated in Fiture 3-2. Compaction means throwing away 

Each segment now has its own in-memory hash table, mapping keys to file offsets. In order to find the value for a key, we first check the most recent segment's hash map; if the key is not present we check the second-most-recent segment, and so on. The merging process keeps the number of segments small, so lookups don't need to check many hash maps.

Lots of detail oges into making this simple idea work in practice. Briefly, some of the issues that are important in a real implementation are:

File format
- CSV is not the best format for a log. It's faster and simpler to use a binary format that first encodes the length of a string in bytes, followed by the raw string
Deleting records
- If you want to delete a key and its associated value, you have to append a special deletion record to the data file. When log segments are merged, the tombstone tells the merging process to discard any previous values for the deleted key.
Crash recovery
- If the databse is restarted, the in-memory hash maps are lost. In principle, you can store each segment's hash map by reading the entire segment file from beginning to end and nothing the offset of the most recent value for every key as you go along. However, that might take a long time if the segment files are large, which would make server restarts painful. Bitcask speeds up recovery by storing a snapshot of each segment's hash map on disk, which can be loaded into memory more quickly.

Partially written records
- The databse may crash at any time, including halfway through appending a record to the log. Bitcask files include checksums, allowing such corrupted parts of the log to be detected and ignored.
Concurrency control
- As writes are appended to the log in a strictly sequential order, a common implementation choice is to have only one writer thread. Data file segments are append-only and otherwise immutable, so they can be read concurrently by multiple threads.

An append-only log seems wasteful at first glance: why don't you update the file in place, overwriting the old value with the new value? But an append-only design turns out to be good for several reasons:
- Appending and segment mergin are sequential write operations, which are generally much faster than random writes, especially on magnetic spinning-disk hard drives. To some extent sequential writes are also preferable on flash-based solid state drives. We will discuss this issue further in "Comparing B-Trees and LSM-Trees"
- Concurrency and crash recovery are much simpler if segment fiels are append-only immutable. For example, you don't have to worry about the case where a crash happended while a value was being overwrittent, leaving you with a file containing part of the old and part of the new value spiced together.
- Merging old segments avoids the problem of data files getting fragmented over time


## SSTables and LSM-Trees

In Figure 3-3, each log-structured storage segment is a sequence of key-value pairs. These pairs appear in the order that they were written, and vlaues later in the log take precedence over values for the same key earlier in the log. Apart from that, the order of key-value pairs in the file does not matter.

Now we can make a simple change to the format of our segment files: we require that the sequence of key-value pairs is sorted by key. At first glance, that requriement seems to break our ability to use sequential writes, but we'll get to that in a moment.

We clal this format Sorted String Table, or SSTable for short. We also require that each key only appears oncce within each merged segment file. SSTables have several big advantages over log segments with hash indexes:
1. Merging segments is simple and efficient, even if the files are bigger than the available memory. The approach is like the one used in the mergesort algorithm and is illustrated in Figure 3-4: you start reading the input files side by side, look at the first key in each file, copy the lowest key to the output file, and repeat. This procudes a new merged segment file, also sorted by key.